In [ ]:
from functions import *
from parameters import *

from sklearn.linear_model import Lasso
from sklearn.linear_model import lasso_path
import torch
import torch.nn.functional as F

from tqdm import tqdm

The raw data, as a CSV file.

In [ ]:
len( np.unique( d['stock_id'] ) )
d.shape

In [ ]:
LOG( "Data (data-frame)" )
filename = "raw/data_ml.csv"
LOG( f"Reading {filename} [20 seconds]" )
d = pd.read_csv(filename)
d['date'] = pd.to_datetime( d['date'] )

predictors = list( signs.keys() )
target = 'R1M_Usd'

Rather than a data-frame, it is often easier to have a list of matrices, one per variable, 
with one row per stock and one column per date: we can easily combine them or apply some function to each row, or each column.

In [ ]:
LOG( "Data (list of matrices)" )
LOG( "Read data/data_ml.pickle" )
dd = load( "data/data_ml.pickle" )

# Single signals
For any of the input variables, we can divide the universe into quintiles, long the top quintile, and short the bottom quintile.

In [ ]:
LOG( "Backtest a single signal" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = predictors[0]
LOG( f"  {signal}" )
x = dd[signal].copy() * signs[signal]
x.fillna(.5, inplace=True)  ## Replace missing values with the median (0.5, since the predictors are uniform)
x = np.where( dd['universe'], 1, np.nan ) * x   # Only invest in stocks in the universe
r = signal_backtest(x, y, date = DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.set_title(signal if signs[signal]>=0 else f"- {signal}")
fig.savefig(f'plots/signal_wealth_{signal}.pdf')
plt.show()

r['performance']

# Lasso
We try to forecast the 1-month forward return from all the input variables. 

As we let the scale of the $L^1$ penalty vary, we have a family of increasingly complex models, from the intercept-only one, to the (unpenalized) least-squares model.

In [ ]:
LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Lasso" )
alphas, coef, _ = lasso_path( X = x, y = y, max_iter = 10_000 )

In [ ]:
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

LOG( "Loop: backtests of the lasso models [5 minutes]" )
res = {}
for j in tqdm(range(len(alphas))):
    signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
    for i,predictor in enumerate(predictors):
        signal += coef[i,j] * dd[predictor].fillna(.5)
    signal = np.where( dd['universe'], 1, np.nan ) * signal
    r = signal_backtest(signal, y, date=DATE1)
    r['performance'  ]['alpha'] = alphas[j]
    r['in-sample'    ]['alpha'] = alphas[j]
    r['out-of-sample']['alpha'] = alphas[j]
    r['performance'  ]['period'] = 'all'
    r['in-sample'    ]['period'] = 'in-sample'
    r['out-of-sample']['period'] = 'out-of-sample'    
    res[ f"{alphas[j]} all" ] = r['performance']
    res[ f"{alphas[j]} in"  ] = r['in-sample']
    res[ f"{alphas[j]} out" ] = r['out-of-sample']
    
    if j == 20:
        # Wealth curves
        fig, ax = plt.subplots()
        for i in range(6):
            ax.plot( r['dates'], r['prices'].iloc[i,:], color = quintile_colours[i] )
        ax.set_yscale('log')
        ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )        
        ax.set_title( f"alpha={alphas[j]:5.3}")
        fig.savefig(f"plots/lasso_wealth_{j}.pdf")
        plt.show()
        
res = pd.concat( res.values() )

We expect the in-sample performance to increase, and the out-of-sample performance to increase and then decrease -- 
this is not what we see here: the out-of-sample performance does not decrease, and a linear model performs extremely well.

In [ ]:
i1 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'in-sample' )
i2 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'out-of-sample' )
i3 = ( res['portfolio'] == 'LS' ) & ( res['period'] == 'all' )
fig, axs = plt.subplots(1,3, figsize=(15,5))
for j,key in enumerate(['Information Ratio', 'CAGR', 'Annualized Volatility']):
    ax = axs[j]
    ax.plot( res[i1]['alpha'], res[i1][key], label = "in-sample" )
    ax.plot( res[i2]['alpha'], res[i2][key], label = "out-of-sample" )
    #ax.plot( res[i3]['alpha'], res[i3][key], label = "all" )
    ax.set_xlabel('Alpha')
    ax.set_xscale('log')
    ax.invert_xaxis()
    ax.set_title(key)
    ax.legend()
fig.savefig("plots/lasso_regularization_path.pdf")
plt.show()

number_of_predictors = ( coef != 0 ).sum(axis=0)
fig, ax = plt.subplots()
ax.plot( alphas, number_of_predictors )
ax.set_xlabel('Alpha')
ax.set_xscale('log')
ax.invert_xaxis()
ax.set_title('Number of predictors')
fig.savefig("plots/lasso_number_of_predictors.pdf")
plt.show()

# Constrained regression

To improve the performance of the linear models, to prevent it from overfitting the data, 
and to ensure the model remains interpretable, we can add constraints on the signs of the coefficients of the regression.
Indeed we know in advance whether each predictor has a positive or negative impact on future returns:
for instance, volatility has a negative impact, while earnings yield has a positive impact.

In [ ]:
from scipy.optimize import lsq_linear

LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Fit the model" )
r = lsq_linear(
    x, y,
    (
        [ 0      if signs[u] > 0 else -np.inf for u in predictors ],
        [ np.inf if signs[u] > 0 else 0       for u in predictors ],
    )
)

# It is not supposed to be that sparse: usually, 2/3 of the coefficients are non-zero...
w = np.round( 1e6 * r.x )
{ p: w[i] for i,p in enumerate(predictors) if w[i] != 0 }

In [ ]:
LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += r.x[i] * dd[predictor].fillna(.5)
    #signal += r_rev.x[i] * dd[predictor].fillna(.5)
    #signal += r_ols.x[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Constrained regression')
fig.savefig("plots/constrained_regression_wealth.pdf")
plt.show()

res['out-of-sample']

# Deep learning: linear model
This is equivalent to the models above, but implemented with a neural network.
* Linear model
* Change the loss function
* Add mini-batches
* Add sign constraints

### Linear model; forecast the returns

In [ ]:
## Data: as above 

LOG( "Training data" )
i = np.array([ str(u) < DATE1 for u in d['date'] ]) 
train = d[i].copy()
x = train[ predictors ]
y = np.log1p(train[ target ])

LOG( "Clean the data" )
i = np.isfinite(y)
x = x[i]
y = y[i]

x = x.fillna(.5)

LOG( "Model" )

class Linear1(torch.nn.Module):
    def __init__(self,k):
        super(Linear1,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,x):
        y = self.linear(x)
        return y

x = torch.tensor(x.values,               dtype=torch.float32)
y = torch.tensor(y.values.reshape(-1,1), dtype=torch.float32)

model = Linear1(x.shape[1])
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

LOG( "Loop" )
for t in tqdm(range(5000)):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

beta = list( model.parameters() )[0]
beta = beta.detach().numpy().flatten()

r = None

LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += beta[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Liner model (via pytorch)')
fig.savefig("plots/model1_linear_wealth.pdf")
plt.show()

res['out-of-sample']

### Linear model; data as id×signal×date array; forecast the returns

Same model as above, but the data is no longer in an (id,date)×signal (2-dimensional) table, but in an id×date×signal 3-dimensional array.

The model and the performance are similar but, for some reason, fitting the model is much more time-consuming.

In [ ]:
LOG( "Training data (3-dimensional array)" )

def get_data_3(all=False):
    """
    Data, as tensors.
    Arguments: all: whether ti return all the data or just the training data
    Returns:   x: id×date×signal
               y: id×date, forward ratio returns
               universe: id×date, 0 or 1
    Global variables used: d, target, predictors, DATE1
    """

    train = data_frame_to_list(d, id_name = 'stock_id', date_name = 'date')
    if all:
        i = np.array( [ True for u in train[ predictors[0] ].columns ] )
    else:
        i = np.array([ str(u) < DATE1 for u in train[ predictors[0] ].columns ])
    train = { k: v.T[i].T for k,v in train.items() }
        #x = train[ predictors ]
        #y = np.log1p(train[ target ])
    y = train[target]
    x = [ train[k] for k in predictors ]
    y = y.values
    x = np.stack( [ u.values for u in x ] )

    y = np.log1p(y)
    universe = np.where( np.isfinite(y), 1, 0 )
    # x = x.fillna(.5)
    x = np.nan_to_num(x, nan=.5)
    y = np.nan_to_num(y, nan=0)

    n = y.shape[0]   # Number of stocks
    l = y.shape[1]   # Number of dates
    k = x.shape[0]   # Number of predictors
    assert k == len(predictors)
    assert n == x.shape[1]
    assert l == x.shape[2]

    # x is now: signal×id×date

    x = x.transpose([1,2,0])
    assert x.shape == (n,l,k) # id×date×signal
    assert y.shape == (n,l)   # id×date
    assert universe.shape == (n,l)

    if False: 
        x = x[:5,:4,:3]
        y = y[:5,:4]
        universe = universe = universe[:5,:4]

    assert ( ~ np.isfinite(x) ).sum() == 0
    assert ( ~ np.isfinite(y) ).sum() == 0
    
    return x, y, universe

x, y, universe = get_data_3()

In [ ]:
class Linear2(torch.nn.Module):
    def __init__(self,k):
        super(Linear2,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,x):
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        y = y[:,:,0]         #  n×l
        return y

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

In [ ]:
model = Linear1(x.shape[2])
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

LOG( "Loop [LONG: 20 minutes for 5000 epochs]" )
N = 5000
losses = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    y_pred = model(x) * universe
    loss = criterion(y_pred,y)
    losses[t] = loss.item()
    pbar.set_description( f"Loss={loss.item():.5f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig, ax = plt.subplots()
ax.plot( losses )
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.set_xscale('log')
fig.savefig("plots/model2_linear_3d_loss.pdf")
plt.show()

In [ ]:
# Backtest the resulting strategy (exactly the same code as above)

beta = list( model.parameters() )[0]
beta = beta.detach().numpy().flatten()

r = None

LOG( "Data for the backtest" )
trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

signal = np.zeros( shape = dd[ list(dd.keys())[0] ].shape )
for i,predictor in enumerate(predictors):
    signal += beta[i] * dd[predictor].fillna(.5)
signal = np.where( dd['universe'], 1, np.nan ) * signal
res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.set_yscale('log')
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_title('Liner model (via pytorch)')
fig.savefig("plots/model2_linear_3d_wealth.pdf")
plt.show()

res['out-of-sample']

### Compute portfolio weights; optimize the information ratio

In [ ]:
x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

In [ ]:
class Linear3(torch.nn.Module):
    def __init__(self,k):
        super(Linear3,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        p = y.exp()             # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p
    
model = Linear3(x.shape[2])
# model( (x,universe) ).detach().numpy().sum(axis=0)   # Should sum to 1 for each date

In [ ]:
## Loop to maximize the IR

LOG( "[LONG] 50 minutes for 10,000 epochs" )

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    w = model( (x,universe) )
    ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={IR.item():.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

fig, ax = plt.subplots()
ax.plot( IRs )
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model3_linear_IR_loss.pdf")
plt.show()

In [ ]:
x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (signal)')
fig.savefig("plots/model3_linear_IR_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights)')
fig.savefig("plots/model3_linear_IR_wealth_weights.pdf")
plt.show()

analyze_returns( r[ r.index > DATE1 ], as_df = True )

### Mini-batches
To make the model more robust, we train it on "mini-batches", defined by a random subset of stocks, and a time interval (contiguous dates).
These are not real mini-batches, because the loss function is not a sum over the observations

In [ ]:
LOG( "[LONG] 30 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear3(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model3b_linear_IR_minibatches_loss.pdf")
plt.show()

In [ ]:
## Wealth curves

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (signal, mini-batches)')
fig.savefig("plots/model3b_linear_IR_minibatches_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights, mini-batches)')
fig.savefig("plots/model3b_linear_IR_minibatches_wealth_weights.pdf")
plt.show()

analyze_returns( r[ r.index > DATE1 ], as_df = True )

### Long-short strategy

In [ ]:
class Linear4(torch.nn.Module):
    def __init__(self,k):
        super(Linear4,self).__init__()
        self.linear = torch.nn.Linear(k,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.linear(x)   # n×l×1
        y = y * universe
        long  = torch.where( y > 0,  y, torch.zeros(1) )
        short = torch.where( y < 0, -y, torch.zeros(1) )
        long  = long [:,:,0]  #  n×l
        short = short[:,:,0]
        long  = long  / ( 1e-6 +  long.sum(axis=0) )
        short = short / ( 1e-6 + short.sum(axis=0) )
        p = long - short
        return p

In [ ]:
## Same code as above

LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear4(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole training sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
fig.savefig("plots/model4_linear_IR_LS_loss.pdf")
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (long-short, signal)')
fig.savefig("plots/model4_linear_IR_LS_wealth.pdf")
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (long-short, weights)')
fig.savefig("plots/model4_linear_IR_LS_wealth_weights.pdf")
plt.show()

analyze_returns( r[ r.index > DATE1 ], as_df = True )   # Very bad...

### TODO: Add transaction costs

In [ ]:
## TODO

# Deep learning: nonlinear model

### Several linear layers

In [ ]:
class Linear6(torch.nn.Module):
    def __init__(self,k):
        super(Linear6,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x)
        y = self.fc2(y)
        y = self.fc3(y)      # n×l×1
        p = y.exp()          # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = Linear6(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (deep linear, signal)')
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (deep linear, weights)')
plt.show()

analyze_returns( r[ r.index > DATE1 ], as_df = True )   

### Nonlinear

In [ ]:
class NonLinear6(torch.nn.Module):
    def __init__(self,k):
        super(NonLinear6,self).__init__()
        self.fc1 = torch.nn.Linear(k,16)
        self.fc2 = torch.nn.Linear(16,4)
        self.fc3 = torch.nn.Linear(4,1)
    def forward(self,xs):
        x, universe = xs
        # x is n×l×k; the linear layer is applied on the last dimension
        y = self.fc1(x); y = F.relu(y)
        y = self.fc2(y); y = F.relu(y)
        y = self.fc3(y)      # n×l×1
        p = y.exp()          # Use a softplus instead of an exponential?
        p = p * universe
        p = p[:,:,0]         #  n×l
        p = p / ( 1e-16 + p.sum(axis=0) )  # portolio weights: positive, sum up to 1 for each date
        return p

In [ ]:
LOG( "[LONG] 25 minutes for 10,000 epochs" )

x, y, universe = get_data_3()

universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

model = NonLinear6(x.shape[2])

optimizer = torch.optim.Adam(model.parameters())
N = 10_000
IRs = np.nan * np.zeros(N)
pbar = tqdm(range(N))
for t in pbar:
    
    x.shape  # id×date×feature
    ## Take half the stocks at random
    i = np.random.choice( x.shape[0], x.shape[0] // 2, replace=False ) 
    ## Take a 3-year period, at random
    j = np.random.choice( x.shape[1] - 36 )
    j = np.arange( j, j+36 )
    
    w = model( (x[i,:,:][:,j,:], universe[i,:,:][:,j,:]) )
    ratio_returns = w * y[i,:,:][:,j,:][:,:,0].expm1()     # y already contains the forward returns
    ratio_returns = ratio_returns.sum(axis=0)
    log_returns = ratio_returns.log1p()
    IR = log_returns.mean() / log_returns.std()
    loss = -IR
    IRs[t] = IR.item()
    pbar.set_description( f"IR={np.nanmean(IRs):.3f}" )
    if not np.isfinite( loss.item() ):
        LOG( f"{t} PROBLEM" )
        break
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
LOG( "DONE" )

## Add the final IR, on the whole sample
w = model( (x,universe) )
ratio_returns = w * y[:,:,0].expm1()     # y already contains the forward returns
ratio_returns = ratio_returns.sum(axis=0)
log_returns = ratio_returns.log1p()
IR = log_returns.mean() / log_returns.std()
IR = IR.item()

## The performance we recorded is very noisy: it is on different periods...
from statsmodels.nonparametric.smoothers_lowess import lowess
fig, ax = plt.subplots()
ax.scatter( 1+np.arange(len(IRs)), IRs )
r = lowess( IRs, 1+np.arange(len(IRs)) )
ax.plot( r[:,0], r[:,1], color = 'black', linewidth=5 )
ax.scatter( 1+len(IRs), IR, color = 'tab:orange', marker='x', s=200, linewidth=5)
ax.set_xlabel("Epoch")
ax.set_ylabel("IR")
ax.set_xscale('log')
plt.show()

In [ ]:
## Wealth curves
##
## This is not the strategy actually learned, but the quintile portfolios from the score.
## The strategy learned provided actual weights.
## 

x, y, universe = get_data_3(all=True)
universe = universe.reshape( y.shape[0], y.shape[1], 1 )
y = y.reshape( y.shape[0], y.shape[1], 1 )
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
universe = torch.tensor(universe, dtype=torch.float32)

signal = model( (x,universe) ).detach().numpy()

trailing_log_returns = LAG( np.log1p( dd[ 'R1M_Usd' ] ) )
y = trailing_log_returns.copy()
y.fillna(0, inplace=True)

assert signal.shape == y.shape
signal = pd.DataFrame( signal, index = y.index, columns = y.columns )

res = signal_backtest(signal, y, date=DATE1)

fig, ax = plt.subplots()
for i in range(6):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (non-linear, signal)')
plt.show()

res['out-of-sample']

In [ ]:
# Backtest the strategy actually learned

r = compute_portfolio_returns( signal, np.expm1(trailing_log_returns) ) 
p = np.exp(cumsum_na(r))               # Log-price = cummulated log-returns
p = replace_last_leading_NaN_with_1(p) # "cumsum" is not the exact inverse of "diff" -- it discards the first value, 1: put it back

fig, ax = plt.subplots()
for i in range(5):
    ax.plot( res['dates'], res['prices'].iloc[i,:], color = quintile_colours[i] )
ax.plot( p.index, p, color='black' )
ax.axvline( pd.to_datetime(DATE1), color='black', linewidth=1 )
ax.set_yscale('log')
ax.set_title('Maximizing the IR (weights)')
plt.show()

analyze_returns( r[ r.index > DATE1 ], as_df = True )   

# Deep learning: lattice networks

In [ ]:
# TODO

# Deep learning: optimization

In [ ]:
# TODO